In [23]:
from network.backbone import ResNet
import matplotlib.pyplot as plt
import numpy as np
from network.region_proposal import RPN, ProposalLayer
from resnet_var_dict import resnet_var_dict
from network.roi_classifier import ROIAlign
import skimage
from skimage import io, transform
import tensorflow as tf
from network.utils import generate_anchors, update_bboxes

In [3]:
tf.enable_eager_execution()

In [64]:
# probs = tf.convert_to_tensor(np.array([[0.9, 0, 0, 0, 0],
#                                        [0, 0, 0, 0.9, 0],
#                                        [0, 0, 0, 0.9, 0],
#                                        [0, 0, 0.9, 0, 0],
#                                        [0, 0, 0.9, 0, 0],
#                                        [0, 0, 0.9, 0, 0]]).astype('float32'))
# roi = tf.convert_to_tensor(np.tile(np.array([[0.1, 0.2, 0.3, 0.4]]).astype('float32'),[6, 1]))
# deltas = tf.convert_to_tensor(np.zeros((6, 4, 4), dtype='float32'))

roi = tf.convert_to_tensor(np.random.uniform(size=(6, 4)).astype('float32'))
large_indices = np.random.randint(0, 5, size=(6))
probs = tf.one_hot(large_indices, 5, on_value=0.9)
deltas = tf.convert_to_tensor(np.random.randn(6, 5, 4).astype('float32'))

class_ids = tf.argmax(probs, axis=1, output_type=tf.int32)
class_inds = tf.stack([tf.range(tf.shape(probs)[0]), class_ids], axis=1)

class_probs = tf.gather_nd(probs, class_inds)
class_deltas = tf.gather_nd(deltas, class_inds)
class_roi = update_bboxes(roi, class_deltas)
class_roi = tf.clip_by_value(class_roi, 0, 1)

# Discard all background and low-confidence boxes
keep = tf.where((class_ids>0) & (class_probs>=0.5))[:, 0]
keep = tf.cast(keep, tf.int32)
class_ids = tf.gather(class_ids, keep)
class_probs = tf.gather(class_probs, keep)
class_roi = tf.gather(class_roi, keep)
unique_class_ids = tf.unique(class_ids)[0]

In [78]:
keep

<tf.Tensor: id=2159, shape=(4,), dtype=int32, numpy=array([0, 1, 4, 5])>

In [79]:
def nms(curr_id):
    indices = tf.where(tf.equal(class_ids, curr_id))[:, 0]
    curr_roi = tf.gather(class_roi, indices)
    curr_probs = tf.gather(class_probs, indices)
    class_keep = tf.image.non_max_suppression(
        curr_roi, curr_probs, 5, 
        iou_threshold=0.3)
    class_keep = tf.gather(indices, class_keep)#tf.gather(keep, tf.gather(indices, class_keep))
    class_keep = tf.cast(class_keep, tf.int32)
    # Pad with -1 so all classes return same size for stacking
    padding = tf.maximum(5-tf.shape(class_keep)[0], 0)
    class_keep = tf.pad(class_keep, [(0, padding)], 'CONSTANT',
                       constant_values=-1)
    class_keep.set_shape([5])
    return class_keep

# Filter ROI for each class with NMS, up to num_detect per class, and put
# in a single list, filtering out -1 padding we inserted in the mapping
nms_keep = tf.map_fn(nms, unique_class_ids, dtype=tf.int32)
nms_keep_2 = tf.reshape(nms_keep, [-1])
nms_keep_2 = tf.gather(nms_keep_2, tf.where(nms_keep_2>-1)[:, 0])
nms_class_probs = tf.gather(class_probs, nms_keep_2)

In [72]:
nms_keep

<tf.Tensor: id=2449, shape=(3, 5), dtype=int32, numpy=
array([[ 0,  1, -1, -1, -1],
       [ 4, -1, -1, -1, -1],
       [ 5, -1, -1, -1, -1]])>

In [80]:
nms_keep_2

<tf.Tensor: id=2576, shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3])>

In [81]:
class_probs

<tf.Tensor: id=2161, shape=(4,), dtype=float32, numpy=array([0.9, 0.9, 0.9, 0.9], dtype=float32)>

In [89]:
new_tens = tf.convert_to_tensor(np.random.uniform(size=(3,4)))

In [113]:
new_tens

<tf.Tensor: id=2588, shape=(3, 4), dtype=float64, numpy=
array([[0.0623067 , 0.07803245, 0.70846297, 0.72388204],
       [0.83712766, 0.78340456, 0.55227725, 0.17387031],
       [0.46995028, 0.81888542, 0.02402927, 0.26616524]])>

In [109]:
num_anch = tf.shape(new_tens)[0]
targets = -1*tf.ones([num_anch], dtype=tf.int32)

In [110]:
ax0 = tf.argmax(new_tens, axis=0, output_type=tf.int32)
ax1 = tf.argmax(new_tens, axis=1, output_type=tf.int32)
max1 = tf.reduce_max(new_tens, axis=1)
targets = tf.where(max1>0.82, tf.zeros([num_anch], dtype=tf.int32), targets)

In [111]:
targets

<tf.Tensor: id=2684, shape=(3,), dtype=int32, numpy=array([-1,  0, -1])>

In [115]:
pos_test = tf.scatter_nd(ax0, tf.ones_like(ax0), tf.shape(targets))
tf.where(pos_test>0)

<tf.Tensor: id=2702, shape=(1, 1), dtype=int64, numpy=array([[1]], dtype=int64)>

In [3]:
backbone = ResNet('channels_last', num_blocks=[3, 4, 6, 3], include_fc=False)
rpn_graph = RPN('channels_last')
proposer = ProposalLayer()

In [4]:
real_picture_1 = io.imread('./example_data/imagenet_tabby_cat_1.jpg').astype('float32')
real_picture_2 = io.imread('./example_data/imagenet_tabby_cat_3.jpg').astype('float32')
input_data = tf.convert_to_tensor(np.stack([real_picture_1, real_picture_2]))

In [5]:
feature_maps = backbone(input_data)
rpn_logits, rpn_probs, rpn_boxes = rpn_graph(feature_maps)

In [54]:
anchors = generate_anchors([32, 64, 128], [0.5, 1, 1.5], 
                           tf.shape(input_data)[1:3],
                           tf.shape(feature_maps)[1:3], 1)
anchors = tf.tile(tf.expand_dims(anchors,0), [tf.shape(input_data)[0], 1, 1])

In [96]:
rois = proposer([rpn_probs, rpn_boxes, anchors])

In [112]:
roi_aligner = ROIAlign()

In [122]:
roi_features = roi_aligner([feature_maps, rois])

In [116]:
roi_features

<tf.Tensor: id=7952, shape=(2, 2000, 7, 7, 2048), dtype=float32, numpy=
array([[[[[ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          ...,
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ]],

         [[ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.61304